In [1]:
from encodec import EncodecModel
from encodec.utils import convert_audio
from mobilenetv3.mobilenetv3 import hswish, hsigmoid, SeModule, Block

import torchaudio
import torch 
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset

import IPython

from datasets import load_dataset, DatasetDict

from sklearn.model_selection import train_test_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Prepare the Data

GTZAN is a good classification dataset for development. It consists of audio/text pairs of music/genre and is a fairly easy task to hit mid-high 90s on given MFCCs or waveforms. This section sets up the dataset. The only reason to run it is if you need to generate encodings at a higher bitrate for further development. The 1.5 bitrate target is already prepared in this repo.

In [ ]:
gtzan = load_dataset("marsyas/gtzan")
encoder = EncodecModel.encodec_model_24khz()

def pre_process_gtzan(gtzan: DatasetDict, target_sr, target_channels) -> (list, list):
    """
    Pre-load the data and process it to the correct sample rate and mono/stereo.
    Returns the pre processed data and a list of the targets.
    """
    data, targets = [], []
    for x in tqdm(gtzan['train']):
        audio, sr = torchaudio.load(x['file'])
        audio = convert_audio(audio, sr, target_sr, target_channels)
        audio = audio.narrow(-1, 0, 718368)
        data.append(audio.unsqueeze(0))
        targets.append(x['genre'])
        
    return data, targets

In [ ]:
data, targets = pre_process_gtzan(gtzan, encode.sample_rate, encode.channels)

In [29]:
def encode_data(data, encoder, batch_size=8, device=None):
    print("Pre-encoding training data")
    
    encodings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(data), batch_size)):
            batch = torch.cat(data[i:i+batch_size], dim=0).to(device)
            encoded_frames = encoder.encode(batch)
            
            codes = torch.cat([e[0] for e in encoded_frames], dim=-1)
            encodings.append(codes)
    
    encodings = torch.cat(encodings, dim=0)
    return encodings

# This takes about 15 minutes to run on a M1 Macbook Pro, a couple of minutes for a GPU
encoder = EncodecModel.encodec_model_24khz()
encoder.set_target_bandwidth(1.5)
encodings = encode_data(data, encoder, batch_size=8, device=device)

/Users/dhuck/.pyenv/versions/3.8.11/envs/thesis/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Pre-encoding training data


  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: unhashable type: 'slice'

In [17]:
print(encodings[0].shape)

torch.Size([8, 2, 2245])


In [18]:
dataset = {
    'data': encodings,
    'targets': targets
}
torch.save(dataset, "gtzan_encodings.pt")

# Load and split the data

In [3]:
class GTZANDataset(Dataset):
    def __init__(self, data, labels, device=None):
        super().__init__()
        self.data = data
        
        self.labels = [torch.tensor(x) for x in labels]
        
        if device is None:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
            data = self.data[index].to(self.device)
            target = self.labels[index].to(self.device)
            
            return data, target

def split_data(data, batch_size=32, random_seed=42, device=None, valid_size=0.1, test_size=0.05, shuffle=True):
    x = data['data']
    y = data['targets']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_seed)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=valid_size, random_state=random_seed)

    train = GTZANDataset(x_train, y_train, device=device)
    valid = GTZANDataset(x_valid, y_valid, device=device)
    test = GTZANDataset(x_test, y_test, device=device)
    
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid, batch_size=batch_size, shuffle=True)
    test_loader  = DataLoader(test,  batch_size=batch_size, shuffle=True)
    return train_loader, valid_loader, test_loader

data = torch.load("./gtzan_encodings-1.5.data")
train, valid, test = split_data(data, batch_size=32)